[![Fixel Algorithms](https://i.imgur.com/AqKHVZ0.png)](https://fixelalgorithms.gitlab.io)

# AI Program

## Exercise 0008 - Deep Learning - Convolution NN for Image Classification

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 20/05/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/Exercise0008.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning


# Deep Learning
import torch
import torch.nn            as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torchinfo
from torchmetrics.classification import BinaryAccuracy
import torchvision
from torchvision.transforms import v2 as TorchVisionTrns

# Miscellaneous
import copy
import gdown
import json
import os
import random
import re
import urllib.request
import shutil
import zipfile

# Typing
from typing import Any, Callable, Dict, List, Optional, Self, Set, Tuple, Union

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
vallToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())


In [ ]:
# Constants

DATA_FOLDER_PATH = 'Data'

# Data Set Links: 
# - MicroSoft Kaggle Cats and Dogs Dataset - https://www.microsoft.com/en-us/download/details.aspx?id=54765 (Dog 11702, Cat 666 might be corrupted)
# - Kaggle Competition - Dogs vs. Cats - https://www.kaggle.com/c/dogs-vs-cats
DATA_SET_URL            = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip'
DATA_SET_FILE_NAME      = 'CatsDogs.zip'
DATA_SET_FOLDER_NAME    = 'CatsDogs'

D_CLASSES  = {0: 'Cat', 1: 'Dog'}
L_CLASSES  = ['Cat', 'Dog']


In [ ]:
# Course Packages

from DataManipulation import DownloadUrl
from DeepLearningPyTorch import TrainModel


In [ ]:
# General Auxiliary Functions

# Class to handle a folder with no labels: Test

from torchvision.datasets.folder import IMG_EXTENSIONS, pil_loader

class TestDataSet( torchvision.datasets.VisionDataset ):
    def __init__(self, root: str = None, transforms: Callable[..., Any] | None = None, transform: Callable[..., Any] | None = None, target_transform: Callable[..., Any] | None = None) -> None:
        super().__init__(root, transforms, transform, target_transform)


        lF = os.listdir(root)
        lFiles = [fileName for fileName in lF if (os.path.isfile(os.path.join(root, fileName)) and (os.path.splitext(os.path.join(root, fileName))[1] in IMG_EXTENSIONS))]

        self.lFiles = lFiles
        self.loader = pil_loader
    
    def __len__(self) -> int:
        
        return len(self.lFiles)
    
    def __getitem__(self, index: int) -> Any:
        
        imgSample =  self.loader(os.path.join(self.root, self.lFiles[index]))
        if self.transform is not None:
            imgSample = self.transform(imgSample)
        
        return imgSample


## Exercise: Cats vs. Dogs

This exercises builds a model based on _Convolutional Neural Network_ for _Binary Image Classification_.  
The data set is from the [Kaggle - Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats) competition.  
The objective is to classify an image either as a _Cat_ or a _Dog_.  

The challenge in this data set is working with images with different dimensions.

The data contains 25,000 RGB images with different dimensions.  

Tasks:
 - Download and arrange data properly.
 - Split data into 22,500 train samples and 2,500 validation samples.
 - Build a dataset and data loader.  
   The data loader must support the case of different image dimensions.
 - Build a parameterized model (Layers, Activations, etc...).
 - Build an optimizer and a scheduler.
 - Build a training loop to optimize hyper parameters.

Tips:
 - Use random transformation to enrich the data set.  
   See [`torchvision.transforms.RandomRotation`](https://pytorch.org/vision/main/generated/torchvision.transforms.RandomRotation.html) as an examples.  
   This is called [_Data Augmentation_](https://en.wikipedia.org/wiki/Data_augmentation).
 - Use [`torchvision.datasets.ImageFolder`](https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html) to load the data easily.
 - Use [`torch.utils.data.random_split`](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split) to split the data set.
 - You may handle the different image dimensions by:
    - Build the model in a manner which is dimension insensitive.
    - Transform the image into a pre defined size (Use padding to keep aspect ratio).
 - 

**Objective**: Above 96% accuracy on the validation set (See the [Competition Leader Board](https://www.kaggle.com/c/dogs-vs-cats/leaderboard)).

* <font color='brown'>(**#**)</font> One may use single output with the Binary Cross Entropy Loss: [`BCELoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html), [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html).

### Handling Different Images Size

There are many options to handle different images size in the same batch / data.  
There are 2 main approaches:

1. Build a Model Insensitive to Image Input  
   Build a model which assumes no knowledge of the image size.  
   Usually it is built by Convolution Layers and Adaptive Pooling so the output is a function of the known number of channels and not the input.  
   The challenge is to handle loading and processing the data.  
   It usually done by padding all images to the size of the largest image.
2. Adapt the Data  
   Apply some combination of padding, resize and crop to have the same image size.

* <font color='brown'>(**#**)</font> For FCN models (1) one could also set the batch size to 1 so each image is on its own. Yet it hurts efficiency greatly.
* <font color='brown'>(**#**)</font> See [PyTorch Forum - How to Create a `dataloader` with Variable Size Input](https://discuss.pytorch.org/t/8278).
* <font color='brown'>(**#**)</font> See [StackOverflow - How to Train Network on Images of Different Sizes with PyTorch](https://stackoverflow.com/questions/72595995).
* <font color='brown'>(**#**)</font> You may use the `TestDataSet` which is adapted to handle no labeled image folder.


In [ ]:
# Parameters

# Data
numSamplesTrain  = 22_500
numSamplesVal    = 2_500
numSamplesValCls = numSamplesVal // 2
calcStat         = False

# Model
dropP = 0.5 #<! Dropout Layer

# Training
imgSize     = 128
batchSize   = 128
numWork     = 2 #<! Number of workers
nEpochs     = 15

# Data Visualization


## Load Data

Load the classification data set.

1. Download the Data Set from [Kaggle Competition - Cat vs. Dog Dataset](https://www.kaggle.com/c/dogs-vs-cats).
2. There are pre defined train and test (Not labeled) split.  
3. Build a script to create a folder `Validation` and move to it part of the labeled data.

In [ ]:
# Download Data (Microsoft - Corrupted)

# if not os.path.isfile(DATA_SET_FILE_NAME):
#     DownloadUrl(DATA_SET_URL, DATA_SET_FILE_NAME)

# if not os.path.isdir(os.path.join(DATA_FOLDER_PATH, DATA_SET_FOLDER_NAME)):
#     oZipFile = zipfile.ZipFile(DATA_SET_FILE_NAME, 'r')
#     lF = oZipFile.namelist()
#     for filePath in lF:
#         filePathExt, fileExt = os.path.splitext(filePath)
#         if (fileExt == '.jpg') or (fileExt == '.jpeg'):
#             if 'Cat' in filePathExt:
#                 oZipFile.extract(filePath, path = os.path.join(DATA_FOLDER_PATH, DATA_SET_FOLDER_NAME, 'Cat'))
#             elif 'Dog' in filePathExt:
#                 oZipFile.extract(filePath, path = os.path.join(DATA_FOLDER_PATH, DATA_SET_FOLDER_NAME, 'Dog'))

In [ ]:
# Build Cat / Dog Folders for Image Folder
# Assumes data is in `.Data/CatsDogs`.
# Data is in Train / Test folder (Extract the inner Zips).

dataSetPath = os.path.join(DATA_FOLDER_PATH, DATA_SET_FOLDER_NAME)
if not os.path.isdir(dataSetPath):
    os.mkdir(dataSetPath)
lFiles = os.listdir(dataSetPath)

if '.processed' not in lFiles: #<! Run only once
    os.makedirs(os.path.join(dataSetPath, 'Validation', 'Cat'), exist_ok = True)
    os.makedirs(os.path.join(dataSetPath, 'Validation', 'Dog'), exist_ok = True)
    for dirName in lFiles:
        dirPath = os.path.join(DATA_FOLDER_PATH, DATA_SET_FOLDER_NAME, dirName)
        if (os.path.isdir(dirPath) and ('train' in dirName.lower())):
            # Process Train Folder
            os.makedirs(os.path.join(dirPath, 'Cat'), exist_ok = True)
            os.makedirs(os.path.join(dirPath, 'Dog'), exist_ok = True)
            for fileName in os.listdir(dirPath):
                fullFilePath = os.path.join(DATA_FOLDER_PATH, DATA_SET_FOLDER_NAME, dirName, fileName)
                _, fileExt = os.path.splitext(fileName)
                fileSize = os.path.getsize(fullFilePath)
                if ((fileSize > 1) and ((fileExt == '.jpg') or (fileExt == '.jpeg'))):
                    if ('cat' in fileName.lower()):
                        shutil.move(fullFilePath, os.path.join(dirPath, 'Cat', fileName))
                    elif ('dog' in fileName.lower()):
                        shutil.move(fullFilePath, os.path.join(dirPath, 'Dog', fileName))
        
            # Should be random, yet for being able to compare
            lF = os.listdir(os.path.join(dirPath, 'Cat'))
            for fileName in lF[-1:-(numSamplesValCls + 1):-1]:
                shutil.move(os.path.join(dirPath, 'Cat', fileName), os.path.join(dataSetPath, 'Validation', 'Cat', fileName))
            lF = os.listdir(os.path.join(dirPath, 'Dog'))
            for fileName in lF[-1:-(numSamplesValCls + 1):-1]:
                shutil.move(os.path.join(dirPath, 'Dog', fileName), os.path.join(dataSetPath, 'Validation', 'Dog', fileName))

    hFile = open(os.path.join(dataSetPath, '.processed'), 'w')
    hFile.close()

In [ ]:
# Data Set 

dsTrain     = torchvision.datasets.ImageFolder(os.path.join(DATA_FOLDER_PATH, DATA_SET_FOLDER_NAME, 'Train'), transform = torchvision.transforms.ToTensor())
dsVal       = torchvision.datasets.ImageFolder(os.path.join(DATA_FOLDER_PATH, DATA_SET_FOLDER_NAME, 'Validation'), transform = torchvision.transforms.ToTensor())
dsTest      = TestDataSet(os.path.join(DATA_FOLDER_PATH, DATA_SET_FOLDER_NAME, 'Test'), transform = torchvision.transforms.ToTensor()) #<! Does not return label
lClass      = dsTrain.classes
numSamples  = len(dsTrain)

print(f'The data set number of samples (Train): {numSamples}')
print(f'The data set number of samples (Validation): {len(dsVal)}')
print(f'The data set number of samples (Test): {len(dsTest)}')
print(f'The unique values of the labels: {np.unique(lClass)}')


### Plot Data

In [ ]:
# Plot Data

vIdx = np.random.choice(numSamples, size = 9)
hF, vHa = plt.subplots(nrows = 3, ncols = 3, figsize = (8, 8))
vHa = vHa.flat

for ii, hA in enumerate(vHa):
    hA.imshow(dsTrain[vIdx[ii]][0].permute((1, 2, 0)).numpy())
    hA.tick_params(axis = 'both', left = False, top = False, right = False, bottom = False, 
                   labelleft = False, labeltop = False, labelright = False, labelbottom = False)
    hA.grid(False)
    hA.set_title(f'Index = {vIdx[ii]}, Label = {L_CLASSES[dsTrain[vIdx[ii]][1]]}')

plt.show()

## Pre Process Data

### Train & Validation Split


In [ ]:
# Calculating the Mean and STD per Channel
# Looping over all files is slow!
# Using Running Mean / Running Squared Mean.
# It would have been faster using Data Loader!

vMean = np.zeros(3)
vSqr = np.zeros(3)


if calcStat:
    for ii, (tImg, _) in enumerate(dsTrain):
        # https://discuss.pytorch.org/t/computing-the-mean-and-std-of-dataset/34949
        vMean += torch.mean(tImg, (1, 2)).numpy()
        vSqr  += torch.mean(torch.square(tImg), (1, 2)).numpy()
    
    vMean /= len(dsTrain)
    vSqr /= len(dsTrain)
    
    # σ = sqrt(E[ (x - μ)^2 ]) = sqrt(E[x^2] - μ^2)
    vStd = np.sqrt(vSqr - np.square(vMean))
else:
    # Pre calculated
    vMean   = np.array([0.48844224, 0.45524513, 0.41706942])
    vStd    = np.array([0.26304555, 0.2565554 , 0.25900563])

In [ ]:
# Define Transformers for Data Augmentation
# Read about PyTorch Transforms: https://pytorch.org/vision/stable/transforms.html.
# Pay attention to v1 vs. v2: 
# - https://pytorch.org/vision/stable/transforms.html#v1-or-v2-which-one-should-i-use.
# - https://pytorch.org/vision/stable/auto_examples/transforms/plot_transforms_getting_started.html#sphx-glr-auto-examples-transforms-plot-transforms-getting-started-py.
# - https://pytorch.org/vision/stable/auto_examples/transforms/plot_transforms_e2e.html#sphx-glr-auto-examples-transforms-plot-transforms-e2e-py.
# Augmentations adds to run time, be careful.


oTransformTrain = TorchVisionTrns.Compose([
    TorchVisionTrns.ToImage(),
    TorchVisionTrns.ToDtype(torch.float32, scale = True),
    TorchVisionTrns.Resize(imgSize),
    TorchVisionTrns.CenterCrop(imgSize),
    TorchVisionTrns.RandomHorizontalFlip(p = 0.5),
    TorchVisionTrns.RandomRotation(5),
    TorchVisionTrns.Normalize(mean = vMean, std = vStd),
])
oTransformVal = TorchVisionTrns.Compose([
    TorchVisionTrns.ToImage(),
    TorchVisionTrns.ToDtype(torch.float32, scale = True),
    TorchVisionTrns.Resize(imgSize),
    TorchVisionTrns.CenterCrop(imgSize),
    TorchVisionTrns.Normalize(mean = vMean, std = vStd),
])

In [ ]:
# Update Transformers

dsTrain.transform   = oTransformTrain
dsVal.transform     = oTransformVal

In [ ]:
# Plot Data
# Updated transformers

vIdx = np.random.choice(numSamples, size = 9)
hF, vHa = plt.subplots(nrows = 3, ncols = 3, figsize = (8, 8))
vHa = vHa.flat

for ii, hA in enumerate(vHa):
    hA.imshow(dsTrain[vIdx[ii]][0].permute((1, 2, 0)).numpy())
    hA.tick_params(axis = 'both', left = False, top = False, right = False, bottom = False, 
                   labelleft = False, labeltop = False, labelright = False, labelbottom = False)
    hA.grid(False)
    hA.set_title(f'Index = {vIdx[ii]}, Label = {L_CLASSES[dsTrain[vIdx[ii]][1]]}')

plt.show()

* <font color='brown'>(**#**)</font> Cropping means some important information might be cropped out.

In [ ]:
# Labels Transform
# Using BCEWithLogitsLoss requires the labels to be probabilities (Float).
# See using Lambda: https://discuss.pytorch.org/t/31857

# oTransformTgt = TorchVisionTrns.Lambda(float) #<! Float64
oTransformTgt = TorchVisionTrns.Lambda(np.float32) #<! Float32
dsTrain.target_transform = oTransformTgt
dsVal.target_transform = oTransformTgt

In [ ]:
# Data Loader

dlTrain = torch.utils.data.DataLoader(dsTrain, shuffle = True, batch_size = 1 * batchSize, num_workers = numWork, persistent_workers = True)
dlVal   = torch.utils.data.DataLoader(dsVal, shuffle = False, batch_size = 2 * batchSize, num_workers = numWork, persistent_workers = True)


## Model 

Defining dimensions insensitive model.
A simple approach is using Fully Convolutional Neural Network (_FCN_) as those layers support arbitrary size of input.  

In [ ]:
# Iterate on the Loader
# The first batch.
tX, vY = next(iter(dlTrain)) #<! PyTorch Tensors

In [ ]:
# Residual Class
# Residual Block:
# - https://scribe.rip/471810e894ed.
# - https://stackoverflow.com/questions/57229054.
# - https://wandb.ai/amanarora/Written-Reports/reports/Understanding-ResNets-A-Deep-Dive-into-Residual-Networks-with-PyTorch--Vmlldzo1MDAxMTk5
# For nn. vs. F. see:
# - https://discuss.pytorch.org/t/31857
# - https://stackoverflow.com/questions/53419474

# Simple Residual Block
class ResidualBlock( nn.Module ):
    def __init__( self, numChnl: int ) -> None:
        super(ResidualBlock, self).__init__()
        
        self.oConv2D1       = nn.Conv2d(numChnl, numChnl, kernel_size = 3, padding = 1, bias = False)
        self.oBatchNorm1    = nn.BatchNorm2d(numChnl)
        self.oReLU1         = nn.ReLU(inplace = True)
        self.oConv2D2       = nn.Conv2d(numChnl, numChnl, kernel_size = 3, padding = 1, bias = False)
        self.oBatchNorm2    = nn.BatchNorm2d(numChnl)
        self.oReLU2         = nn.ReLU(inplace = True) #<! No need for it, 
            
    def forward( self: Self, tX: torch.Tensor ) -> torch.Tensor:
        
        tY = self.oReLU(self.oBatchNorm1(self.oConv2D1(tX)))
        tY = self.oBatchNorm2(self.oConv2D2(tY))
        tY += tX
        tY = self.oReLU(tY)
		
        return tY


In [ ]:
oModel = nn.Sequential(
    nn.Identity(),
    
    nn.Conv2d(3,    16, 3, bias = False), nn.BatchNorm2d(16),  nn.MaxPool2d(2), nn.ReLU(),
    nn.Conv2d(16,   32, 3, bias = False), nn.BatchNorm2d(32),  nn.MaxPool2d(2), nn.ReLU(),
    nn.Conv2d(32,   64, 3, bias = False), nn.BatchNorm2d(64),  nn.MaxPool2d(2), nn.ReLU(),
    nn.Conv2d(64,  128, 3, bias = False), nn.BatchNorm2d(128), nn.MaxPool2d(2), nn.ReLU(),
    nn.Conv2d(128, 256, 3, bias = False), nn.BatchNorm2d(256),                  nn.ReLU(),
    
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(256, 1),
    nn.Flatten(0),
)

torchinfo.summary(oModel, tX.shape, col_names = ['kernel_size', 'output_size', 'num_params'], device = 'cpu')

* <font color='brown'>(**#**)</font> One may use `ResidualBlock` as building blocks.
* <font color='brown'>(**#**)</font> One may try using [PyTorch's _ResNet_](https://pytorch.org/vision/stable/models/resnet.html) model instead.  
  Adjust the input to `224x224`.

## Training the Model

The problem is a binary classification and the output is linear (A logit), hence using `BCEWithLogitsLoss`.

In [ ]:
# Check GPU Availability

runDevice   = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') #<! The 1st CUDA device
oModel      = oModel.to(runDevice) #<! Transfer model to device

In [ ]:
# Set the Loss & Score

hL = nn.BCEWithLogitsLoss() #<! Includes the Sigmoid Built Int
hS = BinaryAccuracy()
hL = hL.to(runDevice)
hS = hS.to(runDevice)

In [ ]:
# Define Optimizer

oOpt = torch.optim.AdamW(oModel.parameters(), lr = 1e-3, betas = (0.9, 0.99), weight_decay = 1e-3) #<! Define optimizer

In [ ]:
# Define Scheduler

oSch = torch.optim.lr_scheduler.OneCycleLR(oOpt, max_lr = 5e-3, total_steps = nEpochs)

In [ ]:
# Train Model

oModel, lTrainLoss, lTrainScore, lValLoss, lValScore, lLearnRate = TrainModel(oModel, dlTrain, dlVal, oOpt, nEpochs, hL, hS, oSch = oSch)

In [ ]:
# Plot Training Phase

hF, vHa = plt.subplots(nrows = 1, ncols = 3, figsize = (12, 5))
vHa = np.ravel(vHa)

hA = vHa[0]
hA.plot(lTrainLoss, lw = 2, label = 'Train')
hA.plot(lValLoss, lw = 2, label = 'Validation')
hA.set_title('Binary Cross Entropy Loss')
hA.set_xlabel('Epoch')
hA.set_ylabel('Loss')
hA.legend()

hA = vHa[1]
hA.plot(lTrainScore, lw = 2, label = 'Train')
hA.plot(lValScore, lw = 2, label = 'Validation')
hA.set_title('Accuracy Score')
hA.set_xlabel('Epoch')
hA.set_ylabel('Score')
hA.legend()

hA = vHa[2]
hA.plot(lLearnRate, lw = 2)
hA.set_title('Learn Rate Scheduler')
hA.set_xlabel('Epoch')
hA.set_ylabel('Learn Rate')

In [ ]:
# Load Best Model
oModelBest = copy.deepcopy(oModel)
oModelBest.load_state_dict(torch.load('BestModel.pt')['Model'])
oModelBest.to('cpu')
oModelBest.eval()

In [ ]:
# Pre PRocess Function
# to apply the same transform on a random image.

def PreProcessImg( tX: torch.Tensor, imgSize: int = imgSize, vMean: np.ndarray = vMean, vStd: np.ndarray = vStd ) -> torch.Tensor:
    # Assumes tX is an image (C x H x W) in range [0, 1]
    tX = TorchVisionTrns.functional.resize(tX, imgSize)
    tX = TorchVisionTrns.functional.center_crop(tX, imgSize)
    tX = TorchVisionTrns.functional.normalize(tX, mean = vMean, std = vStd)
    tX = torch.unsqueeze(tX, 0)

    return tX

In [ ]:
# Evaluate on Test

vIdx = np.random.choice(len(dsTest), size = 9)
hF, vHa = plt.subplots(nrows = 3, ncols = 3, figsize = (12, 12))
vHa = vHa.flat

for ii, hA in enumerate(vHa):
    tImg = dsTest[vIdx[ii]]
    tX = PreProcessImg(tImg)
    valY = oModelBest(tX) #<! Logit -> Label
    # print(f'{valY.item()}')
    lblIdx = int(valY.item() > 0.0)
    hA.imshow(tImg.permute((1, 2, 0)).numpy())
    hA.tick_params(axis = 'both', left = False, top = False, right = False, bottom = False, 
                   labelleft = False, labeltop = False, labelright = False, labelbottom = False)
    hA.grid(False)
    hA.set_title(f'Index = {vIdx[ii]}, Estimated Label = {L_CLASSES[lblIdx]}')

plt.show()